In [116]:
include("framework.jl")
include("create.jl")

mainPath=".."
configPath=mainPath*"/data/config"
outputPath=mainPath*"/data/output"

LoadError: LoadError: ParseError:
# Error @ /home/aarifi/Projects/Capacitance/julia/create.jl:193:9
            theta_ = theta[1]
        else if length(theta) == dimension^3
#       └─────┘ ── use `elseif` instead of `else if`
in expression starting at /home/aarifi/Projects/Capacitance/julia/create.jl:193

In [117]:
config = initConfig();
seed = 0;  Random.seed!(seed);

#cubicArray( dimension, size, config, theta, resolution, radius1, radius2, radius3, fAdd=false )
#cubicArray!( 5, 3, config, 0, 3, 0.5, 0.5, 0.5, true )
cubicArray!( 3, 6, config, collect(0:26), 1, 1.0, 1.0, 1.0, true )
write_csv(config, configPath*"/test.csv")

MethodError: MethodError: Cannot `convert` an object of type Vector{Int64} to an object of type Int64

Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number
   @ Base number.jl:6
  convert(::Type{T}, !Matched::SentinelArrays.ChainedVectorIndex) where T<:Union{Signed, Unsigned}
   @ SentinelArrays ~/.julia/packages/SentinelArrays/1kRo4/src/chainedvector.jl:211
  convert(::Type{T}, !Matched::Number) where T<:Number
   @ Base number.jl:7
  ...


In [118]:
# function read_data( dataLogFile, dataFile )


c, cap = read_data( outputPath, "test")
cap

1×1 Matrix{Float64}:
 5.982142879644378

In [119]:
v = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

v = 2
display( v[2] )

BoundsError: BoundsError: attempt to access Int64 at index [2]